In [ ]:
from instruction import keypair_from_json
from processor import mint_nft
from solana.keypair import Keypair
from solana.rpc.api import Client
from state import ClassEnum
client = Client("https://api.devnet.solana.com")
print("Client is connected" if client.is_connected() else "Client is Disconnected")
payer_keypair = keypair_from_json("./keypair.json")
mints = [Keypair() for _ in range(2)]
print([i.public_key for i in mints])
for mint_keypair in mints:
    print(mint_nft(payer_keypair, mint_keypair, ClassEnum.enum.Serendibite(), client))